# Intro To Neural Networks

This notebook was created as part of udacity deep learning nano degree! see more at [Udacity](https://br.udacity.com/course/deep-learning-nanodegree-foundation--nd101).

## The sigmoid neuron

Sigmoid is an activation for newral networks. Diferent of linear regression that the output for network is 0 or 1 the sigmoid function can have any value between 0 and 1.<br>
It's possible by the sigmoid function as bellow:

$$ sigmoid(x) = 1 / (1 + e^-.^x) $$

$$ y = f(h) = sigmoid(\sum _i w_ix_i + b) $$

In [1]:
import numpy as np

In [2]:
inputs = np.array([0.7, -0.3])
weights = np.array([0.1, 0.8])
bias = -0.1

In [3]:
h = np.dot(weights, inputs) + bias

In [5]:
output = 1 / (1 + np.exp(-h))

In [6]:
output

0.43290709503454572

## Code for gradient descent

### Math for gradient descent

* Update weights: $\Delta w_i = \eta \delta w_i$
* Error term: $\delta = (y - \hat{y}){f}'(h) = (y - \hat{y}){f}'(\sum w_ix_i)$

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

### The data
We are using a dataset school admissions. This dataset has three feature columns, tey are:
* GRE score 
* GPA
* rank of the undergraduate school (numbered 1 through 4)

In [39]:
data = pd.read_csv('./binary.csv')
data.head(3)

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1


### Standardize data

In [40]:
data = pd.get_dummies(data, columns=["rank"])
data.head(3)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0.0,0.0,1.0,0.0
1,1,660,3.67,0.0,0.0,1.0,0.0
2,1,800,4.00,1.0,0.0,0.0,0.0


In [41]:
columns = [ 'gre', 'gpa']
for column in columns:
    mean, std = data[column].mean(), data[column].std()
    data[column] = (data[column] - mean) / std

data.head(3)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0.0,0.0,1.0,0.0
1,1,0.625884,0.736008,0.0,0.0,1.0,0.0
2,1,1.837832,1.603135,1.0,0.0,0.0,0.0


### Splitting dataset into train data and test data

In [42]:
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
train, test = data.ix[sample], data.drop(sample)

In [43]:
train.index

Int64Index([132,  30, 121, 161, 248, 196, 292, 335, 347, 333,
            ...
            104, 385, 359, 241, 197,  38, 300, 301,   8,  70],
           dtype='int64', length=360)

In [44]:
test.index

Int64Index([ 21,  44,  50,  52,  71,  81,  86,  96, 106, 109, 112, 118, 138,
            140, 147, 150, 156, 163, 164, 166, 186, 191, 193, 194, 200, 217,
            218, 220, 222, 252, 254, 284, 290, 295, 314, 315, 326, 346, 348,
            367],
           dtype='int64')

### Splitting data into features and targets

In [45]:
train_features, train_targets = train.drop('admit', axis='columns'), train['admit']

In [46]:
test_features, test_targets = test.drop('admit', axis='columns'), test['admit']

In [47]:
train_features.head(3)

,gre,gpa,rank_1,rank_2,rank_3,rank_4
132,-0.066657,0.026539,0.0,1.0,0.0,0.0
30,-0.412928,1.025050,0.0,0.0,0.0,1.0
121,-0.932334,-1.891652,0.0,1.0,0.0,0.0


In [48]:
train_target.head(3)

145    0
67     0
227    0
Name: admit, dtype: int64

In [49]:
test_features.head(3)

,gre,gpa,rank_1,rank_2,rank_3,rank_4
21,0.625884,0.630901,0.0,1.0,0.0,0.0
44,0.972155,-1.182184,0.0,1.0,0.0,0.0
50,0.452749,1.235263,0.0,0.0,1.0,0.0


In [50]:
test_target.head(3)

0     0
2     1
13    0
Name: admit, dtype: int64

### Neural network implementation

Note: here we are using mean of squared errors to calculate errors.
$$
E = \frac{1}{2m} \sum_\mu (y^\mu - \hat{ y}^\mu)^2
$$

In [53]:
n_records, n_features = train_features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(train_features.values, train_targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # Calculate the output
        h = np.dot(weights, x)
        output = 1 / (1 + np.exp(-h))

        # Calculate the error
        error = (y - output) 

        # Calculate the error term
        error_term = error * (output * (1 - output))

        # Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(train_features, weights))
        loss = np.mean((out - train_targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


('Train loss: ', 0.32215229917945876)
('Train loss: ', 0.21371637809727489)
('Train loss: ', 0.20355666412686951)
('Train loss: ', 0.20059574635994074)
('Train loss: ', 0.19948721628717611)
('Train loss: ', 0.19899520934133877)
('Train loss: ', 0.19874955885615947)
('Train loss: ', 0.1986162111585294)
('Train loss: ', 0.1985393037751616)
('Train loss: ', 0.19849293397565018)


### Testing neural network

In [56]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

tes_out = sigmoid(np.dot(test_features, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == test_targets)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.775
